### 📥 Setups

Installation and Import of required packages

In [774]:
# !pip install -r requirements.txt

In [775]:
import yfinance as yf
import pandas as pd
import numpy as np

Initialization of Analysis Parameters

In [776]:
TICKERS: list[str] = ["AAPL"] # , "MSFT", "AMZN", "TSLA", "NVDA", "JNJ", "JPM", "XOM", "PG", "WMT", "AMT"
START_DATE: str = "2022-01-01"
END_DATE: str = "2025-07-18"

### 🏗️ Data Acquisition

Function for Fetching Daily Price Data

In [777]:
def get_stock_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed adjusted closing price data
    """
    
    data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance and select relevant columns
            df = yf.download(ticker, start=start, end=end, auto_adjust=True)[['Close', 'Volume']]
            df.columns = ['Close', 'Volume']
            
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data found for {ticker} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

### 🧹 Data Cleaning and Preparation

Function for Filling Missing Values and Normalizing Values

In [778]:
def clean_data(data: dict) -> dict:
    """
    Cleans and prepares the stock data for analysis.
    
    Parameters:
        data (dict): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict: Dictionary of DataFrames with cleaned and normalized data
    """

    cleaned_data = {}

    for ticker, df in data.items():
        # Replace zeros in 'Close' with NaN
        df.loc[df['Close'] == 0, 'Close'] = np.nan

        # Forward and backward fill missing values
        df = df.ffill().bfill()
        
        # Normalize the 'Close' prices to a range of 0 to 1
        df['Normalized_Close'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())

        df = df[['Close', 'Close_Normalized', 'Volume']] # Change column order
        
        cleaned_data[ticker] = df

    return cleaned_data

### 📊 Data Analysis

In [779]:
def calculate_moving_average(data: dict, window: int) -> dict:
    """
    Calculates the moving average for each stock's closing price.
    
    Parameters:
        data (dict): Dictionary of DataFrames indexed by ticker symbols
        window (int): Window size for moving average

    Returns:
        dict: Dictionary of DataFrames with moving averages added
    """
    
    ma_data = {}

    for ticker, df in data.items():
        df[f'MA_{window}'] = df['Close'].rolling(window=window).mean()

        ma_data[ticker] = df.dropna(subset=[f'MA_{window}'])

    return ma_data

In [780]:
def calculate_return_and_volatility(data: dict, window: int | None = None) -> dict:
    """
    Calculates the volatility for each stock's closing price.
    
    Parameters:
        data (dict): Dictionary of DataFrames indexed by ticker symbols
        window (int | None): Window size for rolling volatility calculation (Default to None, which is only non-rolling)

    Returns:
        dict: Dictionary of DataFrames with volatility added
    """
    
    volatility_data = {}

    for ticker, df in data.items():
        df['Return'] = df['Close'].pct_change()  # simple % return
        if window is not None:
            df[f'Volatility_{window}'] = df['Return'].rolling(window=window).std()

        df = df.dropna(subset=['Return']) if window is None else df.dropna(subset=[f'Volatility_{window}'])

        volatility_data[ticker] = (df, df['Return'].std())

    return volatility_data